<a href="https://colab.research.google.com/github/marcellosusanto11/predict_ncaa_tournament/blob/master/preproc_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Set working directory
%cd drive/My\ Drive/Colab\ Notebooks/Tugas\ Akhir/

/content/drive/My Drive/Colab Notebooks/Tugas Akhir


In [0]:
# Install package


In [0]:
# Setting general package
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

from matplotlib.pylab import rcParams
# For every plotting cell use this
# fig, axes = plt.subplots()
rcParams['figure.figsize'] = [10,5]
plt.style.use('fivethirtyeight') 
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

## Preproc Dataset
This file is used to compile three different dataset (vegas, kenpom, teamranking) into one final dataset for modelling with categorical response variable 'Result' that represent wether the home team win or not





In [0]:
# Load dataset
df_kenpom = pd.read_csv('Data Kenpom/kenpom.csv')
df_vegas = pd.read_csv('Data LasVegas/vegas.csv')
df_teamrank = pd.read_csv('Data Teamranking/teamranking.csv')
teamname_df = pd.read_csv('Data Lain/change_team_name.csv')

In [5]:
# Overview dataset kenpom
df_kenpom.head()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,AdjustD,AdjustD Rank,AdjustT,AdjustT Rank,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
0,2002,1,Duke,ACC,31,4,1.0,34.19,121.0,1,86.8,1,74.5,8,-0.027,223,9.87,18,109.1,13,99.2,31,6.66,34
1,2002,2,Cincinnati,CUSA,31,4,1.0,30.19,118.1,7,87.9,3,67.4,194,0.002,165,6.58,57,106.3,66,99.7,42,3.48,80
2,2002,3,Maryland,ACC,32,4,1.0,29.25,119.2,4,89.9,7,73.7,15,0.025,104,9.88,16,109.1,11,99.3,33,1.62,120
3,2002,4,Kansas,B12,33,4,1.0,28.99,118.7,5,89.7,6,77.3,3,0.022,109,10.66,10,110.3,4,99.6,40,8.32,23
4,2002,5,Oklahoma,B12,31,5,2.0,26.04,114.9,20,88.9,4,66.5,228,0.043,69,8.77,26,109.0,15,100.2,62,-0.45,169


In [6]:
# Missing values dataset kenpom
df_kenpom.isnull().sum()

Year                  0
Rank                  0
Team                  0
Conference            0
Wins                  0
Losses                0
Seed               2262
Pyth                  0
AdjustO               0
AdjustO Rank          0
AdjustD               0
AdjustD Rank          0
AdjustT               0
AdjustT Rank          0
Luck                  0
Luck Rank             0
SOS Pyth              0
SOS Pyth Rank         0
SOS OppO              0
SOS OppO Rank         0
SOS OppD              0
SOS OppD Rank         0
NCSOS Pyth            0
NCSOS Pyth Rank       0
dtype: int64

Of course variable 'Seed' will have so many missing values because there only a few team in each season competing in the NCAA Tournament 

In [7]:
# Overview dataset vegas
df_vegas.head()

,Date,HomeTeam,AwayTeam,HomeConf,AwayConf,HomeScore,AwayScore,Competition,Odd
0,2011-04-05,St. John's,William & Mary,74,59,Big East,Colonial Athletic Association,Regular Season,-7.5
1,2011-04-05,Mississippi State,Eastern Kentucky,76,66,Southeastern,Ohio Valley,Regular Season,-16.5
2,2011-04-05,Arizona,Valparaiso,73,64,Pacific-12,Missouri Valley,Regular Season,-14.5
3,2011-04-06,St. John's,William & Mary,74,59,Big East,Colonial Athletic Association,Regular Season,-7.5
4,2011-04-06,Mississippi State,Eastern Kentucky,76,66,Southeastern,Ohio Valley,Regular Season,-16.5


Seems like there is a mistake in the order of the vegas dataset. 'Conf' and 'Score variable have their position reversed. So we will change the variable name order

In [0]:
# Change the order of the variable name in vegas dataset
df_vegas.columns = ['Date','HomeTeam','AwayTeam','HomeScore','AwayScore','HomeConf','AwayConf','Competition','Odd']

In [9]:
# Check wether there are duplicated rows
df_vegas[df_vegas.duplicated(subset=list(df_vegas.columns)[1:])]. sort_values('HomeTeam')

,Date,HomeTeam,AwayTeam,HomeScore,AwayScore,HomeConf,AwayConf,Competition,Odd
742,2011-11-11,Air Force,Army,87,71,Mountain West,Patriot League,Regular Season,-12.5
99040,2017-06-11,Alabama,Memphis,82,70,Southeastern,American Athletic Conference,Regular Season,-3.5
41419,2014-06-17,Alabama,Towson,82,54,Southeastern,Colonial Athletic Association,Regular Season,-14.5
105364,2017-09-21,Alabama,Memphis,82,70,Southeastern,American Athletic Conference,Regular Season,-3.5
102388,2017-08-04,Alabama,Memphis,82,70,Southeastern,American Athletic Conference,Regular Season,-3.5
...,...,...,...,...,...,...,...,...,...
68379,2015-09-20,Yale,Fairfield,70,57,Ivy League,Metro Atlantic Athletic,Regular Season,-6.5
61746,2015-06-13,Yale,Fairfield,70,57,Ivy League,Metro Atlantic Athletic,Regular Season,-6.5
63957,2015-07-16,Yale,Fairfield,70,57,Ivy League,Metro Atlantic Athletic,Regular Season,-6.5
64024,2015-07-17,Yale,Fairfield,70,57,Ivy League,Metro Atlantic Athletic,Regular Season,-6.5


From the table above we can see that there are something odd in vegas dataset. For example we can see that there are multiple row showing a match between Yale versus Fairfield where the Score and the Odd are exactly the same, but have a different Date. Logically this phenomenon is impossible to happen so we will drop all these duplicated rows based on all variables except Date

In [10]:
# Drop all duplicated rows in vegas dataset
print('Total rows before dropping :',len(df_vegas))

df_vegas = df_vegas.drop_duplicates(subset=list(df_vegas.columns)[1:])
print('Total rows after dropping :',len(df_vegas))

Total rows before dropping : 160152
Total rows after dropping : 37066


In [11]:
# Missing values dataset vegas
df_vegas.isnull().sum()

Date             0
HomeTeam         0
AwayTeam         0
HomeScore        0
AwayScore        0
HomeConf        71
AwayConf       131
Competition      0
Odd              0
dtype: int64

In [12]:
# See the match in vegas dataset where 'HomeConf' is missing
df_vegas[df_vegas['HomeConf'].isnull()].head()

,Date,HomeTeam,AwayTeam,HomeScore,AwayScore,HomeConf,AwayConf,Competition,Odd
1017,2011-11-21,Chaminade,UCLA,60,92,NaN,Pacific-12,Regular Season,13.5
1022,2011-11-22,Chaminade,Georgetown,61,88,NaN,Big East,Regular Season,22.0
1062,2011-11-23,Chaminade,Tennessee,60,86,NaN,Southeastern,Regular Season,18.0
1089,2011-11-23,Alaska - Anchorage,Murray St.,62,64,NaN,Ohio Valley,Regular Season,13.0
1117,2011-11-25,Alaska - Anchorage,Dartmouth,52,64,NaN,Ivy League,Regular Season,-2.5


In [13]:
# See the match in vegas dataset where 'AwayConf' is missing
df_vegas[df_vegas['AwayConf'].isnull()].head()

,Date,HomeTeam,AwayTeam,HomeScore,AwayScore,HomeConf,AwayConf,Competition,Odd
1115,2011-11-25,Morehead St.,West Alabama,61,52,Ohio Valley,NaN,Regular Season,-16.5
1172,2011-11-26,Bucknell,West Alabama,87,50,Patriot League,NaN,Regular Season,-25.5
1217,2011-11-27,Princeton,West Alabama,66,42,Ivy League,NaN,Regular Season,-19.5
1233,2011-11-27,St. Mary's,San Francisco State,86,52,West Coast,NaN,Regular Season,-34.5
1327,2011-12-02,North Texas,La Sierra University,92,45,Conference USA,NaN,Regular Season,-22.0


After we do some google searching we found that the team in the row where the Conf variable is missing belong to NCAA Division II. Because the purpose of these project is to predict the outcome of NCAA Division I Tournament we will exclude all there rows

In [0]:
# Drop all row with missing values in vegas dataset
df_vegas = df_vegas.dropna()

In [15]:
# Overview dataset teamranking
df_teamrank.head()

,Team,Stat-points-per-game,Last3-points-per-game,Last1-points-per-game,Home-points-per-game,Away-points-per-game,Stat-average-scoring-margin,Last3-average-scoring-margin,Last1-average-scoring-margin,Home-average-scoring-margin,Away-average-scoring-margin,Stat-offensive-efficiency,Last3-offensive-efficiency,Last1-offensive-efficiency,Home-offensive-efficiency,Away-offensive-efficiency,Stat-floor-percentage,Last3-floor-percentage,Last1-floor-percentage,Home-floor-percentage,Away-floor-percentage,Stat-1st-half-points-per-game,Last3-1st-half-points-per-game,Last1-1st-half-points-per-game,Home-1st-half-points-per-game,...,Last3-opponent-effective-possession-ratio,Last1-opponent-effective-possession-ratio,Home-opponent-effective-possession-ratio,Away-opponent-effective-possession-ratio,Stat-win-pct-all-games,Last3-win-pct-all-games,Last1-win-pct-all-games,Home-win-pct-all-games,Away-win-pct-all-games,Stat-win-pct-close-games,Last3-win-pct-close-games,Last1-win-pct-close-games,Home-win-pct-close-games,Away-win-pct-close-games,Stat-opponent-win-pct-all-games,Last3-opponent-win-pct-all-games,Last1-opponent-win-pct-all-games,Home-opponent-win-pct-all-games,Away-opponent-win-pct-all-games,Stat-opponent-win-pct-close-games,Last3-opponent-win-pct-close-games,Last1-opponent-win-pct-close-games,Home-opponent-win-pct-close-games,Away-opponent-win-pct-close-games,Year
0,Oakland,85.0909,94.0000,90.0,88.2222,83.9167,8.54545,16.666700,14.0,16.4444,5.58333,1.14740,1.33602,1.23245,1.19172,1.13083,0.545164,0.630619,0.607902,0.572985,0.534716,41.4848,50.0000,45.0,44.4444,...,0.990525,0.958918,0.965479,0.985963,0.7273,1.0000,1.0,1.0000,0.6250,0.4000,0.0,0.0,1.00,0.0000,0.2727,0.0000,0.0,0.0000,0.3750,0.6000,0.0,0.0,0.00,1.0000,2011
1,Washington,83.7188,72.6667,69.0,90.5000,76.9375,13.46880,6.000000,18.0,20.7500,6.18750,1.12216,1.06575,1.04387,1.19585,1.04632,0.527099,0.503182,0.486048,0.560858,0.492194,39.5938,29.3333,30.0,41.7500,...,0.956001,0.924357,0.911632,0.954951,0.6875,0.6667,1.0,0.8750,0.5000,0.1429,0.5,0.0,0.00,0.1667,0.3125,0.3333,0.0,0.1250,0.5000,0.8571,0.5,0.0,1.00,0.8333,2011
2,VA Military,83.5185,82.6667,81.0,82.7500,84.1333,1.22222,0.333333,-8.0,3.2500,-0.40000,1.07681,1.14043,1.12169,1.05280,1.09649,0.490842,0.524551,0.496313,0.482425,0.497738,39.9630,35.6667,30.0,38.7500,...,1.036790,0.958456,0.978796,0.980017,0.5185,0.6667,0.0,0.5833,0.4667,0.6667,1.0,0.0,1.00,0.5000,0.4815,0.3333,1.0,0.4167,0.5333,0.3333,0.0,0.0,0.00,0.5000,2011
3,LIU-Brooklyn,82.5938,81.3333,85.0,84.1250,81.0625,8.71875,6.666670,3.0,10.5625,6.87500,1.07504,1.08898,1.08229,1.09338,1.05665,0.529290,0.547226,0.554112,0.538522,0.519811,37.8125,36.0000,32.0,38.2500,...,1.004460,1.000000,0.965883,0.947045,0.8438,1.0000,1.0,0.8750,0.8125,0.7778,1.0,1.0,0.75,0.8000,0.1563,0.0000,0.0,0.1250,0.1875,0.2222,0.0,0.0,0.25,0.2000,2011
4,Kansas,82.3030,74.3333,90.0,83.7222,80.6000,17.30300,4.000000,7.0,21.7778,11.93330,1.15198,1.05662,1.27456,1.17982,1.11906,0.553508,0.523077,0.601093,0.562259,0.542944,40.7576,36.6667,43.0,41.2222,...,0.976309,1.014160,0.920928,0.931505,0.9394,1.0000,1.0,0.9444,0.9333,1.0000,1.0,0.0,1.00,1.0000,0.0606,0.0000,0.0,0.0556,0.0667,0.0000,0.0,0.0,0.00,0.0000,2011


In [16]:
# Check wether there are variable with missing values in teamranking dataset
dummy = df_teamrank.isnull().any()
print('Problematic variable :',dummy[dummy==True])

Problematic variable : Series([], dtype: bool)


In [17]:
# Check wether the total of team are the same with the total of rows in each year
df_teamrank.groupby('Year').agg({'Team':'count'}) - df_teamrank.groupby('Year').agg({'Team':'nunique'})

,Team
Year,
2011,0
2012,0
2013,0
2014,0
2015,0
2016,0
2017,0
2018,0
2019,0


In [18]:
# Overview dataset changeteam
teamname_df.head()

,TeamNameSpelling,TeamName
0,abilene chr,abilene chr
1,abilene christian,abilene chr
2,air force,air force
3,akron,akron
4,alabama,alabama


## Preprocessing Team Name Format
We use three dataset from three different source. Because of that the team name format in each dataset differ so first we will synchronize the team name format in all three dataset with a file from Kaggle

First we eill change the team name to lowercase and throw away all the unnecessary white spaces for text preprocessing purpose

In [0]:
# Change team name to lowercase
df_vegas['HomeTeam'] = df_vegas['HomeTeam'].str.lower()
df_vegas['AwayTeam'] = df_vegas['AwayTeam'].str.lower()
df_kenpom['Team'] = df_kenpom['Team'].str.lower()
df_teamrank['Team'] = df_teamrank['Team'].str.lower()
teamname_df['TeamNameSpelling'] = teamname_df['TeamNameSpelling'].str.lower()
teamname_df['TeamName'] = teamname_df['TeamName'].str.lower()

In [0]:
# Throw away unnecessary white spaces
df_vegas['HomeTeam'] = df_vegas['HomeTeam'].str.strip()
df_vegas['AwayTeam'] = df_vegas['AwayTeam'].str.strip()
df_kenpom['Team'] = df_kenpom['Team'].str.strip()
df_teamrank['Team'] = df_teamrank['Team'].str.strip()
teamname_df['TeamNameSpelling'] = teamname_df['TeamNameSpelling'].str.strip()
teamname_df['TeamName'] = teamname_df['TeamName'].str.strip()

Second we will drop all symbol in team name like '.' and '-'

In [0]:
# Change '.' in team name to ''
df_vegas['HomeTeam'] = df_vegas['HomeTeam'].str.replace('\.', '')
df_vegas['AwayTeam'] = df_vegas['AwayTeam'].str.replace('\.', '')
df_kenpom['Team'] = df_kenpom['Team'].str.replace('\.', '')
df_teamrank['Team'] = df_teamrank['Team'].str.replace('\.', '')
teamname_df['TeamNameSpelling'] = teamname_df['TeamNameSpelling'].str.replace('\.', '')
teamname_df['TeamName'] = teamname_df['TeamName'].str.replace('\.', '')

In [0]:
# Change '-' in team name to ' '
df_vegas['HomeTeam'] = df_vegas['HomeTeam'].str.replace('\-', ' ')
df_vegas['AwayTeam'] = df_vegas['AwayTeam'].str.replace('\-', ' ')
df_kenpom['Team'] = df_kenpom['Team'].str.replace('\-', ' ')
df_teamrank['Team'] = df_teamrank['Team'].str.replace('\-', ' ')
teamname_df['TeamNameSpelling'] = teamname_df['TeamNameSpelling'].str.replace('\-', ' ')
teamname_df['TeamName'] = teamname_df['TeamName'].str.replace('\-', ' ')

Third we eill make a dictionary from teamname dataset to synchronize the team name for all three dataset

In [23]:
# Make an identity mapping for each TeamName
list_team = list(teamname_df['TeamName'].unique())
teamname_upd = pd.DataFrame({'TeamNameSpelling':list_team, 'TeamName':list_team})
print('Jumlah row teamname_df awal :',len(teamname_df))

teamname_df = pd.concat([teamname_df, teamname_upd])
teamname_df = teamname_df.drop_duplicates()
print('Jumlah row teamname_df akhir :',len(teamname_df))

Jumlah row teamname_df awal : 895
Jumlah row teamname_df akhir : 840


In [0]:
# Make the dictionary
dict_changeteam = teamname_df.set_index('TeamNameSpelling').to_dict()['TeamName']

In [0]:
# Update the dictionary
dict_team_upd = {'ut rio grande valley':'utrgv', 'southwest missouri st':'missouri st', 'southwest texas st':'texas st'
                ,"st joseph's (pa)":"st joseph's pa", 'chaminade':'chaminade', 'alaska   anchorage':'uaa'
                ,'san francisco state':'san francisco state', 'la sierra university':'la sierra university', 'alabama huntsville':'uah'
                ,'metro state':'metro state', 'west alabama':'west alabama', 'sonoma state':'sonoma state', 'concordia st paul':'concordia st paul'
                ,'merrimack':'merrimack', 'pacific union':'pacific union', 'western new mexico':'wnmu'
                ,'valley city state':'vcsu', 'mayville state':'msu', 'brevard':'brevard'
                ,'southwest state':'smsu', 'western oregon':'wou',
                'evergreen state':'evergreen state', 'hope':'hope', 'hawaii pacific':'hawaii pacific', 'wv wesleyan':'wv wesleyan'}

dict_changeteam.update(dict_team_upd)

We make the update for the dictionary from the remaining team that hasn't been mapped to the new format manually. We identify the team from the last attempt to merge all three dataset

In [0]:
# Change the mismatch in dictionary
# We identify the mismatch from the last attempt to merge all three dataset 
# The faulty name are maryland and loyola md
# Because of the mismatch there are duplicated match in the dataset
dict_changeteam['maryland bc'] = 'umbc'
dict_changeteam['loyola mymt'] = 'loy marymount'

In [0]:
# Change the team name format
df_vegas['HomeTeam'] = df_vegas['HomeTeam'].map(dict_changeteam)
df_vegas['AwayTeam'] = df_vegas['AwayTeam'].map(dict_changeteam)
df_kenpom['Team'] = df_kenpom['Team'].map(dict_changeteam)
df_teamrank['Team'] = df_teamrank['Team'].map(dict_changeteam)

In [28]:
# Check wether there are still a team that hasn't been mapped
idx = df_vegas[df_vegas['HomeTeam'].isnull()].index
print('Problematic team in vegas dataset (Home) :',df_vegas.loc[idx,:]['HomeTeam'].unique())

idx = df_vegas[df_vegas['AwayTeam'].isnull()].index
print('Problematic team in vegas dataset (Away) :',df_vegas.loc[idx,:]['AwayTeam'].unique())

idx = df_kenpom[df_kenpom['Team'].isnull()].index
print('Problematic team in kenpom dataset :',df_kenpom.loc[idx,:]['Team'].unique())

idx = df_teamrank[df_teamrank['Team'].isnull()].index
print('Problematic team in teamranking dataset :',df_teamrank.loc[idx,:]['Team'].unique())

Problematic team in vegas dataset (Home) : []
Problematic team in vegas dataset (Away) : []
Problematic team in kenpom dataset : []
Problematic team in teamranking dataset : []


## Preprocessing Dataset
After we synchronize the team name format we will merge all three dataset to one final dataset for modelling

In [0]:
# Change Date dtype to datetime in vegas dataset
df_vegas['Date'] = pd.to_datetime(df_vegas['Date'])

In [30]:
# Make variabel Year for vegas dataset for merging purpose
# Variable Year represent which year the NCAA Tournament begin
list_year = []
year = 2012
for i in tqdm(range(len(df_vegas)-1)) :
  list_year.append(year)

  # Update the year
  if (df_vegas.iloc[i]['Competition'] == 'NCAA Tournament') & (df_vegas.iloc[i+1]['Competition'] == 'Regular Season') :
    year =  year + 1

# Go
list_year.append(2020)
df_vegas['Year'] = list_year

100%|██████████| 36863/36863 [00:12<00:00, 2913.71it/s]


In [0]:
# Function to make a dataset with variable name format home or away
def change_col(df, jenis) :

  # Initiate supporting variable
  df_final = df.copy()
  list_col = list(df.columns)
  list_col_final = []

  # Iteration to change variable name
  for col in list_col :
    if col not in ['Year'] :
      list_col_final.append(jenis+'_'+col)
    else :
      list_col_final.append(col)

  # Change the variable name in the dataset
  df_final.columns = list_col_final

  return df_final

In [0]:
# Make two dataset with home and away format for kenpom and teamranking dataset
df_kenpom_home = change_col(df_kenpom, 'Home')
df_kenpom_road = change_col(df_kenpom, 'Away')
df_rank_home = change_col(df_teamrank, 'Home')
df_rank_road = change_col(df_teamrank, 'Away')

In [0]:
# Merge the dataset
df_final = df_vegas.merge(df_kenpom_home, left_on=['Year', 'HomeTeam'], right_on=['Year','Home_Team'], how='inner').drop(columns=['Home_Team'])
df_final = df_final.merge(df_kenpom_road, left_on=['Year', 'AwayTeam'], right_on=['Year','Away_Team'], how='inner').drop(columns=['Away_Team'])
df_final = df_final.merge(df_rank_home, left_on=['Year', 'HomeTeam'], right_on=['Year','Home_Team'], how='inner').drop(columns=['Home_Team'])
df_final = df_final.merge(df_rank_road, left_on=['Year', 'AwayTeam'], right_on=['Year','Away_Team'], how='inner').drop(columns=['Away_Team'])

In [0]:
# Order the dataset by Date
df_final = df_final.sort_values('Date')

In [35]:
# Check wether the total match in NCAA Tournament is complete
# One tournament consist of 67 matches
dummy = df_final[df_final['Competition']=='NCAA Tournament']
dummy.groupby('Year').count()['Date']

Year
2012    67
2013    69
2014    67
2015    67
2016    67
2017    68
2018    67
2019    67
Name: Date, dtype: int64

In [0]:
# Get index of problematic rows
list_idx = []

# Check NCAA Tournament 2013
dummy  = df_final[(df_final['Competition']=='NCAA Tournament') & (df_final['Year']==2013)]
list_idx = list_idx + list(dummy.head(2).index)

# Check NCAA Tournament 2017
dummy  = df_final[(df_final['Competition']=='NCAA Tournament') & (df_final['Year']==2017)]
list_idx = list_idx + list(dummy.head(1).index)

We can see that there are surplus matches in 2013 and 2017. After we check the dataset in each year and comparing the Date where the NCAA Tournament begin we can conclude that these surplus matches are not supposed to be NCAA Tournament matches because the match date are before the tournament started. So we will change the value in variable Competition to 'Regular Season' for there problematic matches

In [0]:
# Change the Competition value for problematic rows
df_final.loc[list_idx,'Competition'] = 'Regular Season'

In [38]:
# Check wether the total match in NCAA Tournament are correct
# One tournament consist of 67 matches
dummy = df_final[df_final['Competition']=='NCAA Tournament']
dummy.groupby('Year').count()['Date']

Year
2012    67
2013    67
2014    67
2015    67
2016    67
2017    67
2018    67
2019    67
Name: Date, dtype: int64

Next we will make the response variable 'Result' represented wether the home team win or not

In [0]:
# Make variable 'Result'
def check_winner(hscore, rscore) :
  if hscore > rscore :
    result = 'W'
  else :
    result = 'L'

  return result

df_final['Result'] = df_final.apply(lambda x: check_winner(x['HomeScore'], x['AwayScore']), axis=1)

In [0]:
# Save the final dataset
df_final.to_csv('Data Modelling/data_final.csv', index=False)